<a href="https://colab.research.google.com/github/Soumyajit-Chakraborty/Dyslexia-Detection/blob/main/dyslexia_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile

zippath = "/content/Dyslexia_classification.zip"
extractpath = "/content/dataset"

with zipfile.ZipFile(zippath,'r') as zip_ref:
    zip_ref.extractall(extractpath)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np

IMG_SIZE = (224,224)
BATCH = 16

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    "/content/dataset/Dyslexia_classification",
    validation_split = 0.2,
    subset = "training",
    seed = 42,
    image_size = IMG_SIZE,
    batch_size = BATCH
)

Found 403 files belonging to 2 classes.
Using 323 files for training.


In [ ]:
val_ds = keras.utils.image_dataset_from_directory(
    "/content/dataset/Dyslexia_classification",
    validation_split = 0.2,
    subset = "validation",
    seed = 42,
    image_size = IMG_SIZE,
    batch_size = BATCH
)

Found 403 files belonging to 2 classes.
Using 80 files for validation.


In [ ]:
norm_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y : (norm_layer(x), y))
val_ds = val_ds.map(lambda x, y : (norm_layer(x), y))

In [ ]:
augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

In [ ]:
base_model = keras.applications.MobileNetV2(
    input_shape = IMG_SIZE + (3,),
    include_top = False,
    weights = "imagenet"
)
base_model.trainable = False

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
model = models.Sequential([
    augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1,activation = "sigmoid")
])

model.compile(
    optimizer = keras.optimizers.Adam(1e-4),
    loss = "binary_crossentropy",
    metrics = ["binary_accuracy"]
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 35
)

Epoch 1/35
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - binary_accuracy: 0.5477 - loss: 0.7316 - val_binary_accuracy: 0.5875 - val_loss: 0.6570
Epoch 2/35
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - binary_accuracy: 0.5786 - loss: 0.7041 - val_binary_accuracy: 0.6500 - val_loss: 0.6355
Epoch 3/35
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - binary_accuracy: 0.6626 - loss: 0.6178 - val_binary_accuracy: 0.6875 - val_loss: 0.6112
Epoch 4/35
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - binary_accuracy: 0.6535 - loss: 0.6211 - val_binary_accuracy: 0.7500 - val_loss: 0.5877
Epoch 5/35
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - binary_accuracy: 0.6779 - loss: 0.5986 - val_binary_accuracy: 0.7625 - val_loss: 0.5701
Epoch 6/35
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - binary_accuracy: 0.6494 - loss: 0.6411 - val_binary_accuracy: 0.7625 - val_loss: 0.5515
Epoch 7/35
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - binary_accuracy: 0.6737 - loss: 0.5835 - val_binary_accuracy: 0.7750 - val_loss: 0.5381
Epoch 8/35
2

In [ ]:
model.save("Dyslexia_classifier.h5")